In [72]:
male = open('../data/polish-male-firstnames.txt').read().splitlines()
female = open('../data/polish-male-firstnames.txt').read().splitlines()
words = male + female
words = [w.strip().lower() for w in words]

words[:5]

['jan', 'stanisław', 'andrzej', 'józef', 'tadeusz']

In [73]:
all_chars = sorted(list(set(''.join(words))))
all_chars

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'y',
 'z',
 'ó',
 'ę',
 'ł',
 'ń',
 'ś',
 'ż']

In [74]:
stoi = { s:i+1 for i,s in enumerate(all_chars) }
stoi['.'] = 0
itos = { i: s for s, i in stoi.items() }

In [75]:
size = len(stoi.items())
size

30

In [76]:
import torch
import torch.nn.functional as F

In [97]:
xs, ys = [], []

for w in words:
    w = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(w, w[1:]):
        ich1 = stoi[ch1]
        ich2 = stoi[ch2]
        xs.append(ich1)
        ys.append(ich2)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)
n = xs.nelement()


In [98]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((size, size), generator=g, requires_grad=True)
W

tensor([[ 1.5674e+00, -2.3729e-01, -2.7385e-02, -1.1008e+00,  2.8588e-01,
         -2.9643e-02, -1.5471e+00,  6.0489e-01,  7.9136e-02,  9.0462e-01,
         -4.7125e-01,  7.8682e-01, -3.2843e-01, -4.3297e-01,  1.3729e+00,
          2.9334e+00,  1.5618e+00, -1.6261e+00,  6.7716e-01, -8.4039e-01,
          9.8488e-01, -1.4837e-01, -1.4795e+00,  4.4830e-01, -7.0730e-02,
          2.4968e+00,  2.4448e+00, -6.7006e-01, -1.2199e+00,  3.0314e-01],
        [-1.0725e+00,  7.2762e-01,  5.1114e-02,  1.3095e+00, -8.0220e-01,
         -8.5042e-01, -1.8068e+00,  1.2523e+00, -1.2256e+00,  1.2165e+00,
         -9.6478e-01, -2.3211e-01, -3.4762e-01,  3.3244e-01, -1.3263e+00,
          1.1224e+00,  5.9641e-01,  4.5846e-01,  5.4011e-02, -1.7400e+00,
          1.1560e-01,  8.0319e-01,  5.4108e-01, -1.1646e+00,  1.4756e-01,
         -1.0006e+00,  3.8012e-01,  4.7328e-01, -9.1027e-01, -7.8305e-01],
        [ 1.3506e-01, -2.1161e-01, -1.0406e+00, -1.5367e+00,  9.3743e-01,
         -8.8303e-01,  1.7457e+00,  

In [99]:
for i in range(1000):
    # forward pass
    xenc = F.one_hot(xs, num_classes=size).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims = True)
    loss = - probs[torch.arange(n), ys].log().mean() + 0.01 * (W**2).mean() # negative log likelihood

    # backward pass
    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

    print(loss.item())

3.966998338699341
3.5553791522979736
3.2623214721679688
3.0528297424316406
2.9085543155670166
2.8076632022857666
2.7341630458831787
2.6786868572235107
2.635432004928589
2.600656747817993
2.571925640106201
2.547638416290283
2.5267343521118164
2.5084965229034424
2.4924252033233643
2.4781553745269775
2.4654107093811035
2.453970432281494
2.443657875061035
2.4343249797821045
2.425849199295044
2.4181253910064697
2.411064386367798
2.404590129852295
2.398636817932129
2.3931469917297363
2.3880724906921387
2.383369207382202
2.379000186920166
2.374932289123535
2.3711376190185547
2.3675897121429443
2.3642663955688477
2.361147880554199
2.3582167625427246
2.355457067489624
2.3528552055358887
2.3503975868225098
2.348073959350586
2.3458731174468994
2.3437869548797607
2.341805934906006
2.339923858642578
2.338132381439209
2.336426019668579
2.334799289703369
2.3332464694976807
2.3317625522613525
2.3303439617156982
2.328986406326294
2.327685594558716
2.3264389038085938
2.325242519378662
2.3240935802459717

In [106]:
for i in range(50):
    ix = 0 # start char '.'
    out = ''
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=size).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if(ix == 0):
            break

        out += itos[ix]
    print(out)
        

t
br
bog
awiamiemuro
ad
an
słarkryksłananian
eobiawr
kawiunszefawaław
brariun
bigon
wam
egoliry
etodeolży
d
boldenrebiluoborigd
gest
mianitolicy
miedold
j
jcłafgogiafisenanek
ianisłusłasz
rariadod
dołatastolus
d
jmiulemieliuszbieraw
tadodorantacy
br
rornd
z
lwłan
maw
w
kon
w
w
porartyksz
kty
erarawelian
anz
dyn
wer
elilanalol
tykartun
dibiesłaz
bosław
ekosty
rz
kuszer
gigunonimiejak
